## Analyzing borrowers’ risk of defaulting

This project is to prepare a report for a bank’s loan division. I need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

I'll build a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

Getting Started: Load libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from collections import Counter

Load Dataset

In [2]:
import pandas as pd
data = pd.read_csv('/credit_scoring_eng.csv')
#/datasets/credit_scoring_eng.csv
data.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase


Inspecting the data.


- The “info()” method provides a concise summary of the data; from the output, it provides the type of data in each column, the number of non-null values in each column, and how much memory the data frame is using.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.sample()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6666,0,332038.326832,50,secondary education,1,civil partnership,1,F,retiree,0,24475.636,buy commercial real estate


- The next step is to visually inspect the new data set. There are multiple ways to achieve this:

- The easiest being to request the first few records using the DataFrame data.head()* method. By default, “data.head()” returns the first 5 rows from the DataFrame object df (excluding the header row).
- Alternatively, one can also use “df.tail()” to return the five rows of the data frame.
- For both head and tail methods, there is an option to specify the number of records by including the required number in between the parentheses when calling either method.

In [5]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Some data in days_employed looks not valid. I'll convert days to years check it.

In [66]:
years_employeed = data['days_employed'] / 365
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase,marital_status_reliability,children_status_reliability,income_level,purpose_group
8829,0,359,56,secondary education,1,married,0,M,civil servant,0,29971,housing,0,secondary education,no debt,no debt,average_income,house
2056,0,1728,48,secondary education,1,divorced,3,F,employee,0,16716,profile education,4,secondary education,no debt,no debt,low_income,education
9963,1,2274,28,bachelor's degree,0,married,0,F,business,0,42476,to get a supplementary education,6,bachelor's degree,no debt,no debt,hight_income,education
19460,3,2554,40,SECONDARY EDUCATION,1,married,0,M,employee,0,39281,going to university,6,secondary education,no debt,no debt,hight_income,education
2876,2,4577,37,secondary education,1,married,0,M,employee,0,30190,property,12,secondary education,no debt,no debt,average_income,property & real estate


### Conclusion


- Column ['days_employed'] - contains unreal number of days. For example, 343937.404131 days = 942.294 calendar years. This is simply can't be valid. 

- Column ['days_employed'] - contains negative number of days. It has to be investigated. 

- Column ['education'] - some data in lower case, some in upper plus lower case. In order to make data grouping easy and right, we need to bring everything to one format.

- Column ['purpose'] - the description of the purpose is not consistent: for 1 purpose there're different options, like: 'car purchase',to buy a car','buying my own car'. This has to be fixed and brought to one format.

- If we look at the data info, we could see that such columns as [days_employed] and [total_income] have fewer rows than the entire table: just 19351 from 21525 rows. Which means that 10% of data in those columns are missed. Should investigate the people with missing income. Can they belong to the same category?

- All data errors mentioned above could be because of not an ideal system of filling the blanks.
Usually, clients fill paper blanks manually and after bank employee transfers the data to the system, simply retyping the information into the internal CRM bank system.
As bank employees are dealing with an enormous amount of clients on a daily basis, they as all humans make mistakes, typos, miss the data, or sometimes just leave cells empty in case if client's handwriting was unclear or data need to be verified. 


### Step 2. Data preprocessing

Let's call the value_counts() method, which returns unique values and their counts, in order to determine how many rows are missing in colums.
First let's look at the children column.

In [7]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


We can see that some data has typos: -1, 47 and some data requres futher investigation: 20,76. Can the couples really have 20 kids?

In [8]:
#To continue with data,need to investigate the people with 20 kids. 
#What are their purposes/income? Can they really have 20 kids?
children_20 = data[data['children'] == 20]
children_20

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


<div style="border:solid green 4px; padding: 20px">Looks like the people who are 21 years old or just getting an education, just with 2 years of employment can't have 20 children. In this case let's treat this data as a typo.
Such types of errors in data most likely is a result of manual data enterance mistake.
I'll replace 20 with 2 and -1 with 1.
</div>

In [9]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [10]:
#What about marital status? Is there any weird data?
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [11]:
#What about debt?
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


In [12]:
#What about gender?
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


One row has XNA value. I'll ignore this for now. Since we don't need gender data for our analysis.

In [13]:
#What about clients age?
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


Some clients have age = 0, which is a mistake. It could happen because data was entered manually and the bank employee skipped the row with an age or with the date of birth of the client.

Now let's calculate the number of rows, where clients age = 0 to understand what % of data is missed and decide what to do about it.

In [14]:
print(data.loc[data['dob_years'] == 0].count())

children            101
days_employed        91
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income         91
purpose             101
dtype: int64


Only 101 rows from 21525 don't have info about client's age. As this data is not taken into consideration for futher analysis, I'll not delete or replace these 0's.

Now it is about time to find rows with missing data in the table.

In [15]:
print(data[data['total_income'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


Need to investigate the people with missing income. Can they belong to the same category?

In [16]:
data.loc[data['total_income'].fillna('missing') == 'missing'].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
business,508,0,508,508,508,508,508,508,508,0,508
civil servant,147,0,147,147,147,147,147,147,147,0,147
employee,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105
entrepreneur,1,0,1,1,1,1,1,1,1,0,1
retiree,413,0,413,413,413,413,413,413,413,0,413


<div style="border:solid green 4px; padding: 20px">There's no pattern in the missing total_income. I see nulls don't belong to the same category and 0 values present in every category. The information about income could be missed because the person who entered the data did not know the right value, or missed filling in.

In this case, I have 2 options:

1. Drop None values and lose 10% of our data,

2. Replace missing income with avg income by income_type.

I prefer the second approach as it allows me to keep all the data.
</div>

### Processing missing values

In [17]:
income_avg = data['total_income'].mean()
income_avg

26787.56835465871

Let's replace the missing values with the average income.

In [18]:
data['total_income'] = data['total_income'].fillna(value = income_avg)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Check for nulls in days_employed.

In [19]:
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
dtype: int64


There're 2174 rows of days_employed with 0's. But there's also some data with negative values in this column. In order to fix nulls, I'll need to decide what to do with negative values to positive ones. First I'll count the number of rows with negative values.

In [20]:
print(data[data['days_employed'] < 0].count())

children            15906
days_employed       15906
dob_years           15906
education           15906
education_id        15906
family_status       15906
family_status_id    15906
gender              15906
income_type         15906
debt                15906
total_income        15906
purpose             15906
dtype: int64


15906 from 21525 rows have negative values. I assume that it is a manual error and I'll convert all negative values to positive ones.

In [21]:
#Convert datatype from negative to positive value
data['days_employed'] = data['days_employed'].abs()
data['years_employeed'] = data['days_employed'] / 365
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814


Check how many rows with years_employed data are not valid.

In [22]:
print(data.loc[data['years_employeed'] > data['dob_years']].count())

children            3519
days_employed       3519
dob_years           3519
education           3519
education_id        3519
family_status       3519
family_status_id    3519
gender              3519
income_type         3519
debt                3519
total_income        3519
purpose             3519
years_employeed     3519
dtype: int64


Need to replace these 3519 rows (not valid years and days employeed) with valid data. But first need to investigate if there some category of clients who are affected by wrong days_employeed.

In [23]:
data.loc[data['years_employeed'] > data['dob_years']].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose,years_employeed
income_type,,,,,,,,,,,,
business,18,18,18,18,18,18,18,18,18,18,18,18
civil servant,6,6,6,6,6,6,6,6,6,6,6,6
employee,50,50,50,50,50,50,50,50,50,50,50,50
retiree,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443
unemployed,2,2,2,2,2,2,2,2,2,2,2,2


We see that wrong data affected all categories of clients, and mostly retiree.

In [24]:
#in order to replace employment period, need to find a median for this data. 
#It is important to use median and not mean because 10% number of years_employeed just far away from the reality and average number just will not have any sense.
years_employed_median = data['years_employeed'].median()
years_employed_median

6.0115631969279315

In [25]:
days_employed_median = data['days_employed'].median()
days_employed_median

2194.220566878695

In [26]:
#Change the values that do not make sense
data.loc[(data['years_employeed'] > data['dob_years']), 'years_employeed'] = years_employed_median
data.loc[((data['days_employed']/365) > data['dob_years']), 'days_employed'] = days_employed_median
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,2194.220567,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,6.011563
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087
21521,0,2194.220567,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,6.011563
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347


In [27]:
#Check our data
data.loc[4]

children                              0
days_employed                   2194.22
dob_years                            53
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                F
income_type                     retiree
debt                                  0
total_income                    25378.6
purpose               to have a wedding
years_employeed                 6.01156
Name: 4, dtype: object

Check for nulls in days_employed.

In [28]:
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
years_employeed        0
dtype: int64


Replace 0's with median values.

In [29]:
data['days_employed'] = data['days_employed'].fillna(value=days_employed_median)
print(data[data['days_employed'].isnull()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employeed     0
dtype: int64


In [30]:
data['years_employeed'] = data['years_employeed'].fillna(value=years_employed_median)
print(data[data['years_employeed'].isnull()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employeed     0
dtype: int64


In [31]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,years_employeed
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000
mean,0.479721,2311.541988,43.293380,0.817236,0.972544,0.080883,26787.568355,6.332992
std,0.755528,1977.913079,12.574584,0.548138,1.420324,0.272661,15621.268427,5.418940
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000,0.066141
25%,0.000000,1031.303984,33.000000,1.000000,0.000000,0.000000,17247.708000,2.825490
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,25024.051000,6.011563
75%,1.000000,2508.477456,53.000000,1.000000,1.000000,0.000000,31286.979000,6.872541
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000,50.380685


### Conclusion

<div style="border:solid green 4px; padding: 20px">I investigated if wrong data has some patterns or belongs to a certain categories of clients. I brought data in days_employed column to one format, converting numbers to positive values. Also, I replaced non-valid data with median numbers.</div>

### Data type replacement

I'll convert data type float to int in order to make numbers easier to read. For this purpose I'll use .astype method. DataFrame.astype() comes very handy when we want to case a particular column data type to another data type.

In [32]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['years_employeed'] = data['years_employeed'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
 12  years_employeed   21525 non-null  int32 
dtypes: int32(3), int64(5), object(5)
memory usage: 1.9+ MB
None


In [33]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,23
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,11
2,0,5623,33,Secondary Education,1,married,0,M,employee,0,23341,purchase of the house,15
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,11
4,0,2194,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,6


### Conclusion

<div style="border:solid green 4px; padding: 20px">I converted data with numbers to int format in order to make it easy to read and analyze. I've checked that all colums have the right data format and I can continue working with them.</div>

### Processing duplicates

Next step in to search for duplicate data.

In [34]:
print(data['education'].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64


I will change everything to lowercase.

In [35]:
data['education_lowercase'] = data['education'].str.lower()
print(data['education_lowercase'].value_counts())

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education_lowercase, dtype: int64


I  want to identify duplicate purpose rows.

In [36]:
# output True if row above is the same
data.purpose.duplicated()

0        False
1        False
2         True
3        False
4        False
         ...  
21520     True
21521     True
21522     True
21523     True
21524     True
Name: purpose, Length: 21525, dtype: bool

In [37]:
type(data.duplicated())

pandas.core.series.Series

In [38]:
#how manu duplicated rows in the table?
data.duplicated().sum()

54

In [39]:
# examine duplicated rows
data.loc[data.duplicated(), :].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase
2849,0,2194,41,secondary education,1,married,0,F,employee,0,26787,purchase of the house for my family,6,secondary education
4182,1,2194,34,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,26787,wedding ceremony,6,bachelor's degree
4851,0,2194,60,secondary education,1,civil partnership,1,F,retiree,0,26787,wedding ceremony,6,secondary education
5557,0,2194,58,secondary education,1,civil partnership,1,F,retiree,0,26787,to have a wedding,6,secondary education
7808,0,2194,57,secondary education,1,civil partnership,1,F,retiree,0,26787,having a wedding,6,secondary education


In [40]:
# keep='first'
# mark duplicates as True except for the first occurence
data.loc[data.duplicated(keep='first'), :].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase
2849,0,2194,41,secondary education,1,married,0,F,employee,0,26787,purchase of the house for my family,6,secondary education
4182,1,2194,34,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,26787,wedding ceremony,6,bachelor's degree
4851,0,2194,60,secondary education,1,civil partnership,1,F,retiree,0,26787,wedding ceremony,6,secondary education
5557,0,2194,58,secondary education,1,civil partnership,1,F,retiree,0,26787,to have a wedding,6,secondary education
7808,0,2194,57,secondary education,1,civil partnership,1,F,retiree,0,26787,having a wedding,6,secondary education


In [41]:
# keep='last'
# this is useful for splitting the data
data.loc[data.duplicated(keep='last'), :].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase
120,0,2194,46,secondary education,1,married,0,F,employee,0,26787,university education,6,secondary education
520,0,2194,35,secondary education,1,civil partnership,1,F,employee,0,26787,to have a wedding,6,secondary education
541,0,2194,57,secondary education,1,married,0,F,employee,0,26787,second-hand car purchase,6,secondary education
554,0,2194,60,secondary education,1,married,0,M,employee,0,26787,buy real estate,6,secondary education
680,1,2194,30,bachelor's degree,0,married,0,F,civil servant,0,26787,purchase of the house for my family,6,bachelor's degree


In [42]:
data.drop_duplicates().shape

(21471, 14)

### Conclusion

- I indentified and propped 54 rows with duplicates. I decided to keep last part of dublicated rows in the table, because after I dropped 54 row, the other 54 are no longer duplicated.


- Possible reasons for the appearance of duplicates in the data:

- Lack of data standardization. The text fields for entering clients data vary across system databases. While one system might provide separate fields for a record, others may just have one field, which permits users to enter clients information however they want.

- Lack of client ID. Effectively matching clients requires clients ID numbers, but if records don't have unique data points to match, multiple clients may be associated with the same record.

- Default values. Text fields that don't have to be filled, can reduce accurate clients matching.

### Categorizing Data

To find relevant data, we need to rely on categorization—organizing the selected data into groups according to specific criteria.
To analyze the data I'll split the clients by groups.

Create a function, which will help to categorize clients by 2 paramenters: marital status category and presence of debt.

In [43]:
def marital_status_debt (row):
    marital_status = row['family_status']
    debt = row['debt']

    if marital_status=='married':
        if debt== 1:
            return 'married with debt'

    if marital_status=='civil partnership':
        if debt== 1:
            return 'civil with debt'

    if marital_status=='unmarried':
        if debt== 1:
            return 'unmarried with debt'
        
    if marital_status=='divorced':
        if debt== 1:
            return 'divorced with debt'
        
    if marital_status=='widow / widower':
        if debt== 1:
            return 'widow with debt'

    return 'no debt'

data['marital_status_reliability'] = data.apply(marital_status_debt, axis=1)

Create a function, which will help to categorize clients by 2 paramenters: number of children and presence of debt.

In [44]:
def children_status_debt (row):
    children_status = row['children']
    debt = row['debt']

    if children_status == 1:
        if debt== 1:
            return '1 child and debt'

    if children_status == 2:
        if debt == 1:
            return '2 children and debt'

    if children_status == 3:
        if debt == 1:
            return '3 children and debt'
        
    if children_status > 3:
        if debt== 1:
            return 'more than 3 children and debt'
    if children_status == 0:
        if debt== 1:
            return 'no child and debt'

    return 'no debt'

data['children_status_reliability'] = data.apply(children_status_debt, axis=1)

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

First I'll check how many clients have children and what % of then have debt.

Using groupby method I'll calculate the number of clients in each category: 0 children, 1, 2,..., n children.

In [45]:
children_group = data.groupby('children')['debt'].count()
children_group

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: debt, dtype: int64

Now I'll write a children_status_debt (row) function, that splits clients with debt to categories according to the number of children per client.

Now I'll calculate default rate per each category of clients with children and without them. For this purpose I'll use pivot table.

In [46]:
data.pivot_table(index='children', values= 'debt', aggfunc =['sum','count', 'mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14149,0.075129
1,445,4865,0.091470
2,202,2131,0.094791
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


### Conclusion

<div style="border:solid green 4px; padding: 20px">There's no big gap between clients who have debt and 1 or 2 children: 9 - 9.5%. Clients with 3 and more children have a default_rate little bit lower: 8.2%. It proves that there's no big difference in how many children client have and the probability that he/she will have debt. 

But there's 7,5% default rate of clients with no children who have debt. It shows that clients without children are more likely to return the debt than clients with children.</div>

- Is there a relation between marital status and repaying a loan on time?

First I'll check how many clients fall for every marital status category.

In [47]:
family_status_grouped = data.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_grouped

debt     
                   count  sum
family_status                
civil partnership   4177  388
divorced            1195   85
married            12380  931
unmarried           2813  274
widow / widower      960   63

In [48]:
family_status_grouped.sum()

debt  count    21525
      sum       1741
dtype: int64

1741 clients from total 21525 have debt. It gives us 8% clients who don't pay their obligations to bank.

Second I'll create a pivot table, which will help to calculate a default rate for every category of clients, divided by family_status.

In [49]:
data.pivot_table(index='family_status', values= 'debt', aggfunc =['sum','count', 'mean'])

,sum,count,mean
,debt,debt,debt
family_status,,,
civil partnership,388,4177,0.092890
divorced,85,1195,0.071130
married,931,12380,0.075202
unmarried,274,2813,0.097405
widow / widower,63,960,0.065625


### Conclusion

<div style="border:solid green 4px; padding: 20px">The highest default rate of debt has unmarried category of clients - 9.7% and clients in civil partnership 9.3%, second place is shared between married 7.5% and divorced 7.1% people. The lowest debt rate goes to widows 6.6%.</div>

- Is there a relation between income level and repaying a loan on time?

I will split clients by income levels. First let's find min, max and avg income.

In [50]:
income_level_grouped = data.agg({'total_income': ['min', 'max', 'mean']})
income_level_grouped

,total_income
min,3306.000000
max,362496.000000
mean,26787.064065


I want to investigate the category of clients who fall into different income levels. What is the reason for low and hight income?

In [51]:
income_log = data[['family_status','income_type', 'total_income', 'debt', 'years_employeed','education_lowercase']] 
income_log.sort_values(by='total_income', ascending=False)

,family_status,income_type,total_income,debt,years_employeed,education_lowercase
12412,married,business,362496,0,4,bachelor's degree
19606,married,business,352136,1,7,bachelor's degree
9169,civil partnership,employee,276204,0,14,secondary education
20809,unmarried,employee,274402,0,12,secondary education
17178,civil partnership,business,273809,0,15,bachelor's degree
...,...,...,...,...,...,...
14276,married,retiree,3503,0,6,secondary education
1598,civil partnership,retiree,3471,0,6,secondary education
16174,married,employee,3418,0,9,secondary education
13006,civil partnership,retiree,3392,0,6,secondary education


In order to make table more readable, I'll group the income_type into a table according to total_income and find the average rating.

In [52]:
income_log.groupby('income_type').mean().sort_values('total_income',ascending=False)

,total_income,debt,years_employeed
income_type,,,
entrepreneur,53326.500000,0.000000,3.500000
business,31826.918977,0.073943,5.355556
civil servant,27287.184373,0.058944,8.489376
employee,25916.412087,0.095422,5.894055
retiree,22459.045124,0.056017,6.000000
unemployed,21014.000000,0.500000,6.000000
student,15712.000000,0.000000,1.000000
paternity / maternity leave,8612.000000,1.000000,9.000000


As I can see from the data above, under low income category 3 types of income:unemployed, student and paternity / maternity leave. The hight level of income covers only 2 types: entrepreneur and business. Others go under average level.

What we need is categorization—combining the data into categories.   

I’ll group the clients as follows:


—Clients with income under 22000 will go into the low income category.


—Clients with income between 22000 and 31000 (inclusively) will go into the average income category.


—Clients with over 31000 will go into the hight income category.

In [53]:
def income_level(total_income):
    if total_income <= 22000:
        return 'low_income'
    if total_income <= 31000:
        return 'average_income'
    return 'hight_income'

In [54]:
#test the function
print(income_level(30000))

average_income


The function is working correctly. The next thing in order is to create a separate column for income categories and document them in its cells.
To do this, I'll call the apply() method.

In [55]:
data['income_level'] = data['total_income'].apply(income_level)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase,marital_status_reliability,children_status_reliability,income_level
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,23,bachelor's degree,no debt,no debt,hight_income
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,11,secondary education,no debt,no debt,low_income
2,0,5623,33,Secondary Education,1,married,0,M,employee,0,23341,purchase of the house,15,secondary education,no debt,no debt,average_income
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,11,secondary education,no debt,no debt,hight_income
4,0,2194,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,6,secondary education,no debt,no debt,average_income


Now let's get income_level groups with the value_counts() method:

In [56]:
data['income_level'].value_counts()

low_income        8814
average_income    7212
hight_income      5499
Name: income_level, dtype: int64

I'll calculate default_rate for each income level clients category.

In [57]:
data.pivot_table(index='income_level', values= 'debt', aggfunc =['sum','count', 'mean'])

,sum,count,mean
,debt,debt,debt
income_level,,,
average_income,620,7212,0.085968
hight_income,395,5499,0.071831
low_income,726,8814,0.082369


### Conclusion

<div style="border:solid green 4px; padding: 20px">According to the investigation, the customers with an average level of income are more possible to have debt in the future: 8.6%. 

Surprisingly customers with a low level of income more likely to return debt on time 8.2%. 

Customers with a high level of income have the lowest default rate: 7.2%.</div>

- How do different loan purposes affect on-time repayment of the loan?

First I'll check how many purposes is in the database.

In [58]:
data['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
housing                                     647
purchase of the house                       647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

I see that an employee who was managing the Bank database didn't create a consistent short list with unique purposes.

Instead he/she was adding the purpose reasons manualy and didn't have system to manage them. 

In order to be able to categorize clients by credit reasons I'll tokenize row['purpose] and Counter(lemmas). 

Then, I'll get the most popular words and basing on them build the categories of purposes.

In [59]:
wordnet_lemma = WordNetLemmatizer()

For preprocessing a dataframe, I'll loop the tokenization procedure over all data['purpose'] rows.

In [60]:
def purpose_reason(row):
    sentence = row['purpose']
    words = nltk.word_tokenize(sentence)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    new = []
    for w in words:
        w_token = word_tokenize(w)
        if w_token != '':
            new.append(w_token)
    return Counter(lemmas)
               
               

data['purpose_group'] = data.apply(purpose_reason, axis = 1)
             
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employeed,education_lowercase,marital_status_reliability,children_status_reliability,income_level,purpose_group
10958,0,2194,53,secondary education,1,unmarried,4,F,retiree,0,15362,housing,6,secondary education,no debt,no debt,low_income,{'housing': 1}
7479,0,2194,31,some college,2,unmarried,4,F,civil servant,0,26787,purchase of the house for my family,6,some college,no debt,no debt,average_income,"{'purchase': 1, 'of': 1, 'the': 1, 'house': 1,..."
13264,0,1342,54,secondary education,1,married,0,F,employee,0,65881,buy real estate,3,secondary education,no debt,no debt,hight_income,"{'buy': 1, 'real': 1, 'estate': 1}"
17836,0,13372,59,secondary education,1,married,0,M,civil servant,0,47419,second-hand car purchase,36,secondary education,no debt,no debt,hight_income,"{'second-hand': 1, 'car': 1, 'purchase': 1}"
14683,2,1478,40,some college,2,married,0,F,employee,0,17120,university education,4,some college,no debt,no debt,low_income,"{'university': 1, 'education': 1}"


In [61]:
count = data['purpose_group'].sum()
count

Counter({'purchase': 3314,
         'of': 2998,
         'the': 1288,
         'house': 1908,
         'car': 4315,
         'supplementary': 909,
         'education': 3114,
         'to': 3081,
         'have': 774,
         'a': 5130,
         'wedding': 2348,
         'housing': 1912,
         'transaction': 2610,
         'having': 777,
         'for': 1294,
         'my': 2396,
         'family': 641,
         'buy': 2367,
         'real': 4478,
         'estate': 4478,
         'commercial': 1315,
         'residential': 607,
         'construction': 635,
         'own': 2240,
         'property': 2542,
         'building': 1246,
         'buying': 1637,
         'second-hand': 968,
         'with': 1281,
         'become': 412,
         'educated': 412,
         'getting': 869,
         'an': 443,
         'ceremony': 797,
         'get': 447,
         'higher': 426,
         'profile': 436,
         'university': 949,
         'renting': 653,
         'out': 653,
         'ren

Now I can group purposes into categories:
1. house
2. car
3. education
4. wedding
5. property & real estate

In [62]:
purpose_key_words = [
    ['house', 'house, housing, purchase, house'],
    ['car', 'car, second-hand'],
    ['education', 'education, university, educated'],
    ['wedding', 'wedding, having a wedding, ceremony'],
    ['property & real estate', 'real, estate, property, real estate']
]

And create function def purpose_cleaned to assign categoriesed purposes to each row according to key words found on that row.

In [63]:
def purpose_cleaned(row):
    kew_words = row['purpose']       
    words = nltk.word_tokenize(kew_words)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]    
    for w in lemmas:       
        for row in purpose_key_words:                                     
            if w in row[1] and len(w) > 2:                    
                return row[0] 
               
data['purpose_group'] = data.apply(purpose_cleaned, axis = 1)
#cheking if the function works
data.loc[:, ['purpose', 'purpose_group']].sample(20)

,purpose,purpose_group
11840,going to university,education
1132,having a wedding,wedding
11279,purchase of the house for my family,house
16518,buying my own car,car
4241,buy real estate,property & real estate
13481,car,car
5795,buy commercial real estate,property & real estate
1001,transactions with my real estate,property & real estate
21309,buying property for renting out,property & real estate
1788,to become educated,education


Next step is to calculate the number of cients in each purpose category.

In [64]:
data['purpose_group'].value_counts()

property & real estate    7020
house                     4275
education                 4022
car                       3860
wedding                   2348
Name: purpose_group, dtype: int64

Now I'll calculate default_rate for each purpose category.

In [65]:
data.pivot_table(index='purpose_group', values= 'debt', aggfunc =['sum','count', 'mean'])

,sum,count,mean
,debt,debt,debt
purpose_group,,,
car,353,3860,0.091451
education,370,4022,0.091994
house,306,4275,0.071579
property & real estate,526,7020,0.074929
wedding,186,2348,0.079216


### Conclusion

<div style="border:solid green 4px; padding: 20px">The most reliable clients with a purpose related to buying or renovating houses, this category has default rate = 4.4%, second place in reliability rating goes to clients who's purpose to have a wedding, least reliable clients with a purpose to buy a car or get an education.</div>

### Step 4. General conclusion


- In order to prevent data duplication Bank should improve its data storage system: use clients' IDs and return to a user system error, when he/she enters negative values to the rows where it is not acceptable: income, days employed, age, etc.


- As was found, the highest default rate of debt has an unmarried category of clients. To improve this situation the Bank could add a condition to its loan agreement for such category: to have a guarantor for expensive purchases.


- With regards to the different types of purpose and their correlation to the debt, I found that 'buying a car' and 'education' have a less reliable rating, comparing to other purposes. It could happen due to the fact that education takes a long time to pay back. Sometimes finding a new job with a high salary takes longer than the loan deadlines.
Car is a that type of expense, which didn't bring investments back. Unlike real estate, education, or property.  And people tend to overestimate their creditworthiness in the pursuit of comfort. In order to improve the credit payback system for this category of clients Bank should create a step-by-step procedure for assessing customer creditworthiness. Evaluate the thresholds on amounts per client, depending on the assessment of a group of factors, such as years of employment, average income per last year, availability of deposits in the Bank, additional income, marital status, credit history, etc.


- There's up to 2% difference in debt_default rate between clients who have children and clients who don't. Clients who don't have children are more likely to return money to the Bank. It dosen't mean that Bank should reject on loans to clients with children. It only proves, that during creditworthiness eveluation process, Bank should add more steps. For example, check if second parent (if any) has good credit history or other factors confirming his solvency.


- Clients with average levels of income have the lowest credibility, default rate = 8.6%, clients with low income have a bit better default rate = 8.1%, and the most trustworthiness clients with high-income level = 7.2%.  The explanation for this is that people with average income tend to buy more expensive things that they can afford: buying houses, cars, getting prestigious educations. In order to improve credit rating for clients with average income, Bank should check whether the client has credits in other banks, his/her credit history, and have a threshold for a loan, which could be calculated as a % of annual income.

### Project Readiness Checklist

I Put 'x' in the completed points to check myself.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.